# Comparing satellite data with models

Henk Eskes (henk.eskes@knmi.nl) <br>
September 2024


<h2>Comparing models and observations</h2>

The difference between a measurement and the model-simulation of the measurement is a key quantity in data assimilation and validation, and is often called the <b>departure</b>:

<div> <img src="images/dept.png" width="400"> </div>

Note: This has the unit of the observations, e.g. ppb, µg/m<sup>3</sup>, mol/m<sup>2</sup>.

Relative differences do not have a unit and are often easier to understand and interpret for non-specialist .<br>
These can be expressed as percentage difference (%). 

<div> <img src="images/rel_diff.png" width="550"> </div>

A more symmetric form is also possible (useful when some observations become close to, or equal to 0):

<div> <img src="images/rel_diff_sym.png" width="450"> </div>

In the CAMS validation reports most results are expressed in relative units, <br>
such as the normalised bias, fractional gross error (a normalised absolute mean difference) or the correlation.


Model output is in most cases four dimensional (4D), e.g. 3D fields produced every hour, written as a 4D matrix  

<div> <img src="images/x4d.png" width="50"> </div>

while the observation <i>y</i> is taken at a given point in space and time. <br>
Therefore we need an “observation operator” <i>H</i>, 
to computing the model equivalent of the observation at the location and time of the observation.<br>
The departure becomes:

<div> <img src="images/Hx_y.png" width="120"> </div>

In it’s simplest form <i>H</i> is an interpolation in time, followed by a three-dimensional interpolation in space

<div> <img src="images/H_spacetime.png" width="320"> </div>


For <b>remote sensing</b> (with satellites or with ground instruments) the retrieval product  is often a <b>profile</b> provided on a vertical retrieval grid</b>. , e.g. an ozone profile. <br>
For instance, the TROPOMI instrument has an ozone profile product, specified on a number of vertical levels.

The 3D interpolation operator can then be written as a successive horizontal lat-lon interpolation followed by a  vertical interpolation from the model levels to the retrieval grid:   

<div> <img src="images/H_4d.png" width="600"> </div>



<h3>Comparing models and remote-sensing retrievals</h3>

It seems natural to use the above form of the observation operator (based on space-time interpolations) in comparisons against satellite profiles,  but <i>this is not the optimal way</i> to compare model output with remote sensing observations.

Why not?

1. Remote sensing observations can not retrieve concentrations at one point (vertical level),  but rather provide height-integrated averages of concentrations weighted by  sensitivity profiles. Comparing with model ozone point values is like comparing ozone apples with ozone pears.

2. The number of independent pieces of information in the measurements is often much less  that the number of retrieval layers. In this case the retrieval profile shapes are constrained  or “smoothed” by  a-priori profile information (e.g. fixed profile shape or climatology),  and the comparisons will partly reflect the a-priori input. For instance, retrievals 


<h3> Optimal Estimation Theory </h3>

The bible of remote sensing retrieval theory is the book
"Inverse methods for atmospheric sounding"
by Clive Rodgers
World Scientific, 2000.

The concepts which are introduced briefly below are derived and discussed in detail in this book. 
It is a must read for people working on (satellite) retrievals.

<div> <img src="images/Rodgers_book.png" width="200"> </div>


<h3>Comparing models and remote-sensing retrievals (continued)</h3>

Instead of comparing with retrievals we can use the model and observation operator to <i>simulate radiances</i> or reflectances (as function of wavelength) and compare directly with the satellite observations of radiation:

<div> <img src="images/Fx_r.png" width="320"> </div>

Where <i>F</i> is a forward radiative transfer operator which computes wavelength-dependent  radiances given a model trace gas or aerosol vertical profile (and other quantities  that determine the radiances).

As shortcut we wrote the profile at the satellite location as:

<div> <img src="images/HIx4d_is_x.png" width="120"> </div>

Notes:

- A  major advantage of the radiance comparison approach: There is no a-priori profile shape dependence in this comparison. Retrieval methods generally depend on a pre-defined a-priori profile shape. In contrast, a forward radiative transfer model does not.

- In numerical weather prediction the assimilation of satellite radiances is common practice, for instance to assimilate temperature profiles.  Also in CAMS first experiments are conducted to assimilate radiances, for instance to constrain aerosol properties.

- For validation this is not always practical: differences in radiances are not always so easily related  to errors in trace gas concentrations.  Also: a complex forward model to compute the radiances has to be  implemented in the comparison.



We have now discussed two options:

1. Compare retrieved profiles with a model using a 4D interpolation observation operator.

2. Comparing measured satellite radiances with radiances simulated with the model and a radiative transfer model.

But there is a third option:

Instead of comparing with retrievals products we can also reproduce the retrieval.  This means:

- first simulate radiances (as function of wavelength) and then <br>
- apply the retrieval operator with the a-priori profile and other parameters used for the retrieval.

This may sounds very complicated, but it is not, as explained below.

The model-observation difference now becomes:

<div> <img src="images/RF_Rr.png" width="300"> </div>

Where <i>F</i> is a forward radiative transfer operator computing radiances based on the model profile and input parameters. <br>
<i>R</i> is the retrieval method which depends on:

- The radiances or reflectances <i>r</i> 

- The a-priori profile <b>x</b><sub><i>a</i></sub>. In this tutorial these are trace gas profiles, for instance ozone or NO<sub>2</sub>.

- Other retrieval input parameters <b>b</b>. <br>   Examples of “b” are surface albedo, cloud properties, water vapour, temperature profile

Forward and retrieval operators are typically non-linear, but can be linearised around the a-priori, 
see e.g. the book of Rodgers (2000):

<div> <img src="images/FR_lin.png" width="350"> </div>

Where <b>F</b> is a matrix converting from real space to wavelength space, 
and <b>R</b> is the linearised retrieval matrix connecting wavelength space to real space.

The model-simulated retrieval 

<div> <img src="images/RF.png" width="500"> </div>

The product of the linearised retrieval method times forward model 

<div> <img src="images/A_RF.png" width="80"> </div> 

is the <b>Averaging Kernel</b> of the retrieval (see Rodgers, 2000). 

Excercise: derive this relation using the formulas above.

The model-observation difference (departure) now becomes:

<div> <img src="images/ak_eq.png" width=220"> </div>

where <b>y</b>
is the retrieved quantity (for instance an ozone profile, a vector quantity).

The averaging kernel and a-priori profile are normally <b>included in the retrieval dataproduct</b>. <br>
Therefore this method 3 comparison can be  done without additional radiative transfer calculations.

Note:

In the discussion so far the averaging kernel is normally a square matrix.<br>
However, this is not needed: the forward model may use a different set of vertical layers  than the retrieval method.

For an accurate forward modelling of the radiation a sufficient number of layers is needed. Typically 20-40 layers are used.

But the number of independent pieces of information derived from the satellite data is often (much) smaller than the number of layers. For instance, the ozone profile retrieval for TROPOMI has about 6 pieces of information on ozone. Therefore the results could be provided for a limited number of layers or subcolumns.

The extreme example is the retrieval of total columns (like for TROPOMI NO2), where the retrieval produces  just one number instead of a profile. <br>
The averaging kernel then becomes a vector .

Typically remote sensing retrievals are very sensitive to the a-priori profile shape  provided to the retrieval. However, the averaging kernel depends in the same way  on the a-priori, because it contains the retrieval method. While absolute comparisons  still depend on the a-priori, the relative comparison

<div> <img src="images/rel_diff.png" width="550"> </div>

becomes a-priori independent when the averaging kernel is used in the comparison. Note that this holds for weak absorbers with a linear relation between the absorber and the radiance (such as for NO2 to a good approximation). For strong absorbers the forward model becomes non-linear and has to be linearised around an a-priori starting point. This introduces a weak dependence of the comparison on the linearisation point.  

There is also a high-level equivalence between comparisons in radiance space and  in profile space. When an optimised profile            reproduces the radiances, eq. (2),  then the same profile will give a perfect match with the retrieval,  independent of the a-priori chosen in the retrieval method.

But:

- The forward and retrieval operators should be reasonably linear around the a-priori. 

- The retrieval method should not be too simple and should not loose information.

Reference: Stefano Migliorini, "On the Equivalence between Radiance and Retrieval Assimilation", MWR 2012.

<h3>Satellite retrievals of columns</h3>

Several column products, like TROPOMI SO2, HCHO and NO2 total columns, are derived using the <b>Differential Optical Absorption Spectroscopy</b> method. This two-step approach first compares differential spectral features in the measurement with reference absorption spectra, to derive the amount of absorption along the light path. Secondly, a so-called air-mass factor is derived using radiative transfer calculations to compute the vertical column.

The DOAS column retrieval approach has been re-formulated using Rodgers Optimal Estimation formalism (Eskes and Boersma, 2003). 
This leads naturally to the definition of averaging kernel vectors for total column DOAS retrieved quantities.

<div> <img src="images/Eskes2003.png" width="600"> </div>

NO2 retrieval example: The DOAS averaging kernel profiles are a measure of the sensitivity of the measurement to 
NO2 at a given altitude. NO2 is retrieved in the 400-500 nm spectral range, where Raileigh scattering Each retrieval has its own kernel, depending on retrieval parameters like the surface reflectivity, cloud cover, geometry (solar zenith and satellite viewing angles). The image below shows three examples:

- When the surface is dark the sensitivity close to the surface (and kernel value) is small.

- When the surface is bright (e.g. sand, snow) the sensitivity close to the surface is large.

- Clouds obscure the NO2 close to the surface. The satellite has almost no sensitivity below the cloud, but is more sensitive just above the cloud.

<div> <img src="images/no2_kernel_examples.png" width="400"> </div>

For the weak absorber total column retrieval (e.g. TROPOMI NO2) the formula to compare model with the retrieval simplifies further. For this retrieval 

<div> <img src="images/xa_axa.png" width="150"> </div>

and therefore the (model-observation) departure becomes

<div> <img src="images/Ax_xhat.png" width="100"> </div>

The retrieved column (a single number) is compared with the dot product of the kernel vector and the vector of modelled partial columns for all the vertical layers.<br>
Note that the a-priori profile is not needed to compare the retrieval with the model. <br>
This is the reason why the a-priori is not provided in the NO2 data product.

<h3>Error sources in the model-satellite comparison</h3>

The spread in (model-obs) values is determined by three error terms: <br>
- Model (forecast) errors  <br>
- Combined observation + retrieval errors <br>
- Representativity errors

<div> <img src="images/error_terms.png" width="500"> </div>

In this formula we assume that the three error terms are uncorrelated.

The satellite retrieval provides an error covariance matrix quantifying the retrieval errors. This is a matrix: because of the limited vertical resolution the errors in the retrieved values will be correlated between layers at different altitudes. If the retrieved ozone has a positive error at a certain altitude, say 20 km, then it is likely that the error at 21 km is also positive. This is described by the off-diagonal terms in the retrieval covariance matrix. Layers further apart may be anti-correlated: a positive error at 20 km may on average be related to a negative error at 40 km altitude. 

Representativity errors are often neglected but are also often the largest term, especially for strongly varying short-lived species!<br>
Examples:<br>
- Comparing a roadside NO2 / PM monitor with the 10x10 km gridcell of a European air quality model <br>
- Difference in satellite footprint and model gridcell (different air masses) <br>
- Measurements in mountainous terrain

If kernels are not used in the comparison, then a so-called "smoothing error" (Rodgers, 2000) has to be added to the error budget to reflect the finite (vertical) resolution of the retrieval.

<h3>Take-home messages</h3>

- Retrievals (profiles, columns) are often strongly dependent on the a-priori used in the retrieval method.

- Remote sensing retrievals can be compared with models in two ways: (1) by using only the 4D interpolation form of the observartion operator, or (2) by applying the averaging kernels after the 4D interpolation.

- Method (2) is highly recommended. By using the averaging kernels the relative comparison between model and retrieval becomes independent (or only weakly dependent in the non-linear case) on the a-priori assumptions made in the retrieval. In general (model-measurement) differences will be smaller with method 1 because the smoothing error is avoided.

